1. In this programming problem and the next you'll code up the greedy algorithms from lecture for minimizing the weighted sum of completion times..

Download the text file below.

**jobs.txt**


This file describes a set of jobs with positive and integral weights and lengths.  It has the format

[number_of_jobs]

[job_1_weight] [job_1_length]

[job_2_weight] [job_2_length]

...

For example, the third line of the file is "74 59", indicating that the second job has weight 74 and length 59.

You should NOT assume that edge weights or lengths are distinct.

Your task in this problem is to run the greedy algorithm that schedules jobs in decreasing order of the difference (weight - length).  Recall from lecture that this algorithm is not always optimal.  IMPORTANT: if two jobs have equal difference (weight - length), you should schedule the job with higher weight first.  Beware: if you break ties in a different way, you are likely to get the wrong answer.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below. 

ADVICE: If you get the wrong answer, try out some small test cases to debug your algorithm (and post your test cases to the discussion forum).

2. For this problem, use the same data set as in the previous problem.

Your task now is to run the greedy algorithm that schedules jobs (optimally) in decreasing order of the ratio (weight/length).  In this algorithm, it does not matter how you break ties.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below. 

In [3]:
from collections import defaultdict
from collections import Counter

class job_schedule:

    def __init__(self, job_weights, job_lengths):

        self.job_weights = job_weights
        self.job_lengths = job_lengths

        self.job_schedule = []

        self.cost = 0
        self.completion_time = 0

    def minus_rule(self):

        self.cost = 0

        values = []
        for i in range(len(self.job_weights)):
            
            values.append(self.job_weights[i] - self.job_lengths[i])

        self.job_schedule = list(zip(self.job_weights, self.job_lengths, values))

        self.job_schedule.sort(key=lambda x: (x[-1], x[0]), reverse=True)

        # duplicate_values = [key for key, val in Counter(values).items() if val > 1]

        # # print(duplicate_values)
        # for val in duplicate_values:
            
        #     tmp_idx = [idx for idx, (a, b, c) in enumerate(self.job_schedule) if c==val]

        #     # print(tmp_idx)
        #     # print(self.job_schedule[tmp_idx[0]:tmp_idx[-1]+1])
        #     self.job_schedule[tmp_idx[0]:tmp_idx[-1]+1] = sorted(self.job_schedule[tmp_idx[0]:tmp_idx[-1]+1], key=lambda x: x[0], reverse=True)

    def ratio_rule(self):
        
        values = []
        for i in range(len(self.job_weights)):
            
            values.append(self.job_weights[i]/self.job_lengths[i])
        
        self.job_schedule = list(zip(self.job_weights, self.job_lengths, values))
        self.job_schedule.sort(key=lambda x: x[-1], reverse=True)

    
    def calc_cost(self):
        
        self.completion_time = 0
        self.cost = 0

        for weight, length, _ in self.job_schedule:

            self.completion_time += length

            self.cost += weight * self.completion_time


if __name__ == "__main__":
    
    txt_file = './jobs.txt'

    job_weights = []
    job_lengths = []
    with open(txt_file, 'r') as f:
        for line in f.readlines()[1:]:
            nums = line.split()
            # print(nums)
            job_weights.append(int(nums[0]))
            job_lengths.append(int(nums[-1]))

    # print(job_weights, job_lengths)  
    print("Weights: ", len(job_weights))  
    print("Lengths: ", len(job_lengths))


    job_schedule_test = job_schedule(job_weights, job_lengths)
    job_schedule_test.minus_rule()
    job_schedule_test.calc_cost()

    print("Minus total cost is: ", job_schedule_test.cost)

    job_schedule_test.ratio_rule()
    job_schedule_test.calc_cost()

    print("Ratio total cost is: ", job_schedule_test.cost)

    #print(job_schedule_test.job_schedule)
    
    # Weights:  10000
    # Lengths:  10000
    # Minus total cost is:  69119377652
    # Ratio total cost is:  67311454237

Weights:  10000
Lengths:  10000
Minus total cost is:  69119377652
Ratio total cost is:  67311454237


3. In this programming problem you'll code up Prim's minimum spanning tree algorithm.

Download the text file below.

**edges.txt**

This file describes an undirected graph with integer edge costs.  It has the format

[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874. 

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Your task is to run Prim's minimum spanning tree algorithm on this graph.  You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative --- in the box below. 

IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn) time implementation of Prim's algorithm should work fine. OPTIONAL: For those of you seeking an additional challenge, try implementing a heap-based version. The simpler approach, which should already give you a healthy speed-up, is to maintain relevant edges in a heap (with keys = edge costs).  The superior approach stores the unprocessed vertices in the heap, as described in lecture.  Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [5]:
from collections import defaultdict
import heapq


class Graph:

    def __init__(self, n):

        self.num_node = n
        self.nodes = set(range(1, n+1))
        self.edges = defaultdict(list)
        self.costs = defaultdict()
    
    def add_edge(self, from_node, to_node, cost):
        self.edges[from_node].append(to_node)
        self.costs[from_node,to_node] = cost


def mst_prim(graph, source_node = 1):

    X = set()
    X.add(source_node)
    MST = defaultdict(list)

    cost = 0

    while X != set(graph.nodes):

        cand_edge_costs = []

        for u_node in X:

            cand_edge_costs += [(u_node,v_node,graph.costs[u_node,v_node]) for v_node in graph.edges[u_node] if v_node not in X]
        
        min_edge = min(cand_edge_costs, key=lambda x: x[-1])

        # print(min_edge)

        MST[min_edge[0]].append(min_edge[1])
        
        X.add(min_edge[1])
        
        cost += min_edge[-1]

    return cost

def mst_prim_heap(graph, source_node = 1):

    q, visited = [(0, source_node, ())], set()

    total_cost = 0

    while visited != set(graph.nodes):

        (cost, v1, path) = heapq.heappop(q)

        if v1 not in visited:
            visited.add(v1)
            path = (v1, path)

            total_cost += cost

            for v2 in graph.edges[v1]:
                if v2 in visited:
                    continue

                heapq.heappush(q, (graph.costs[v1, v2], v2, path))

    return total_cost

if __name__ == "__main__":
    

    txt_file = './edges.txt'

    graph = Graph(500)

    with open(txt_file, 'r') as f:

        for line in f.readlines()[1:]:
            nums = line.split()
            ##### pay attention to here, the graph is undirected, so the edges should be added twice
            graph.add_edge(int(nums[0]), int(nums[1]), int(nums[2]))
            graph.add_edge(int(nums[1]), int(nums[0]), int(nums[2]))

    # print(graph.edges)


    print(mst_prim(graph))

    print("Resp:", mst_prim_heap(graph))
    
    #Resp: -3612829

-3612829
Resp: -3612829
